# Setup

In [1]:
from consav.runtools import write_numba_config
write_numba_config(disable=0,threads=20,threading_layer='omp')

In [2]:
import time
import numpy as np
from scipy.io import savemat

import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
colors = [x['color'] for x in plt.style.library['seaborn']['axes.prop_cycle']]
markers = ['s','P','D','v','^','*']

%load_ext autoreload
%autoreload 2

from consav.misc import elapsed
from TwoAssetModelCont import TwoAssetModelContClass

# Comparison with MATLAB code

**MATLAB code:** https://github.com/JeppeDruedahl/HANK

The numbers used below is generated by runing `Compare_with_Python.m`

**Setup model:**

In [3]:
t0 = time.time()
model = TwoAssetModelContClass(name='HANK',like_HANK=True)
print(f'model created in {elapsed(t0)}')

model created in 10.4 secs


**Create incomeprocess data used in MATLAB:**

In [4]:
import income_process
grid_z_log,_,z_markov,z_dist = income_process.construct_jump_drift(model.par)   
savemat('incomeprocess.mat',{'logygrid':grid_z_log.reshape(33,1),
                             'ydist':z_dist.reshape(33,1),
                             'ymarkov':z_markov})

**Solve:**

In [5]:
model.solve()

Grids created in 2.1 secs
Solution prepared in 2.5 secs
Solving HJB:
    1: 84.1540116083472753
    2: 36.4077721627507245
    3: 30.0339429262052171
    4: 29.2756855009812398
    5: 19.8659501420693374
    6: 18.3467315209349096
    7: 14.4465947306913449
    8: 12.1943889342600045
    9: 10.3557107522892551
  100: 0.0000064547207899
 converged in 23.7 secs in iteration 143

Solving KFE:
    1: 3073.0191025923722918
    2: 4082.3379225562857755
    3: 973.1641277035087114
    4: 643.7900170138345857
    5: 443.0360652919839595
    6: 285.2451558790876334
    7: 227.0242440494948823
    8: 147.4079026078866264
    9: 120.5089814762711740
  100: 0.0003001026948368
  200: 0.0000000079083718
 converged in 8.1 secs secs in iteration 294


**Compare with MATLAB results:**

In [6]:
model.calculate_moments()

print(f"A_supply in Python: {model.moms['A_supply']:.8f}")
print(f'A_supply in MATLAB: {20.309410438823377:.8f}')
print(f"N_supply in Python: {model.moms['N_supply']:.8f}")
print(f'N_supply in MATLAB: {0.578669074537820:.8f}')
print(f"B_demand in Python: {model.moms['B_demand']:.8f}")
print(f'B_demand in MATLAB: {1.618428960125360:.8f}')
print(f'Fraction of poor HtM in Python: {model.moms["frac_b0_a0"]:.3f}')
print(f'Fraction of poor HtM in MATLAB: {0.103:.3f}')
print(f'Fraction of wealthy HtM in Python: {model.moms["frac_b0_apos"]:.3f}')
print(f'Fraction of wealthy HtM in MATLAB: {0.186:.3f}')
print(f'Fraction of borrowers in Python: {model.moms["frac_bneg"]:.3f}')
print(f'Fraction of borrowers in MATLAB: {0.146:.3f}')

print('')
model.show_moments()

A_supply in Python: 20.30941050
A_supply in MATLAB: 20.30941044
N_supply in Python: 0.57866907
N_supply in MATLAB: 0.57866907
B_demand in Python: 1.61842896
B_demand in MATLAB: 1.61842896
Fraction of poor HtM in Python: 0.103
Fraction of poor HtM in MATLAB: 0.103
Fraction of wealthy HtM in Python: 0.186
Fraction of wealthy HtM in MATLAB: 0.186
Fraction of borrowers in Python: 0.146
Fraction of borrowers in MATLAB: 0.146

Equilibrium objects:
 rb: 0.0050
 ra: 0.0142
 w: 1.814
 Pi: 0.171
 capital-labor discrepancy: -0.00285850
 profit discrepancy: 0.01961031

Aggregates:
 GDP: 1.743
 capital-labor ratio: 28.246
 capital-output ratio: 2.345
 bond-output ratio: 0.232

Fractions:
 poor HtM: 0.103 [0.103, 0.146]
 wealthy HtM: 0.186 [0.186, 0.279]
 borrowers: 0.146 [0.146, 0.133]

Iliquid wealth:
 top   0.1: 0.071
 top     1: 0.409
 top    10: 0.877
 bottom 50: 0.001
 bottom 25: 0.000
 gini: 0.820

Liquid wealth:
 top   0.1: 0.024
 top     1: 0.182
 top    10: 0.750
 bottom 50: -0.032
 bottom

**Save:**

In [7]:
t0 = time.time()
model.save()
print(f'model saved in {elapsed(t0)}')

model saved in 0.1 secs


**Load:**

In [8]:
t0 = time.time()
HANK = TwoAssetModelContClass(name='HANK',load=True)
print(f'model loaded in {elapsed(t0)}')

model loaded in 17.8 secs


In [9]:
HANK.show_moments()

Equilibrium objects:
 rb: 0.0050
 ra: 0.0142
 w: 1.814
 Pi: 0.171
 capital-labor discrepancy: -0.00285850
 profit discrepancy: 0.01961031

Aggregates:
 GDP: 1.743
 capital-labor ratio: 28.246
 capital-output ratio: 2.345
 bond-output ratio: 0.232

Fractions:
 poor HtM: 0.103 [0.103, 0.146]
 wealthy HtM: 0.186 [0.186, 0.279]
 borrowers: 0.146 [0.146, 0.133]

Iliquid wealth:
 top   0.1: 0.071
 top     1: 0.409
 top    10: 0.877
 bottom 50: 0.001
 bottom 25: 0.000
 gini: 0.820

Liquid wealth:
 top   0.1: 0.024
 top     1: 0.182
 top    10: 0.750
 bottom 50: -0.032
 bottom 25: -0.032
 gini: 0.862

Consumption: avarage = 1.146, gini = 0.311
Value: avarage = -19.049


# Find equilibrium

In [10]:
model = TwoAssetModelContClass()
model.find_ra(KN0=28.15752068,Pi0=0.17394494,tol=1e-5) # using good starting values

  1: KN = 28.15752068, Pi = 0.17394494
    implied ra = 0.0142
    implied w = 1.8142
    implied KN = 28.1571 [diff.: 0.00001594]
    implied Pi = 0.1739 [diff.: -0.00000462]
    time: 27.2 secs
  2: KN = 28.15747579, Pi = 0.17394486
    implied ra = 0.0142
    implied w = 1.8142
    implied KN = 28.1579 [diff.: -0.00001452]
    implied Pi = 0.1739 [diff.: 0.00000456]
    time: 9.0 secs
  3: KN = 28.15751666, Pi = 0.17394494
    implied ra = 0.0142
    implied w = 1.8142
    implied KN = 28.1571 [diff.: 0.00001322]
    implied Pi = 0.1739 [diff.: -0.00000384]
    time: 8.8 secs
  4: KN = 28.15747943, Pi = 0.17394487
    implied ra = 0.0142
    implied w = 1.8142
    implied KN = 28.1578 [diff.: -0.00001204]
    implied Pi = 0.1739 [diff.: 0.00000378]
    time: 8.8 secs
  5: KN = 28.15751333, Pi = 0.17394494
    implied ra = 0.0142
    implied w = 1.8142
    implied KN = 28.1572 [diff.: 0.00001097]
    implied Pi = 0.1739 [diff.: -0.00000319]
    time: 8.5 secs
  6: KN = 28.15748245, P

**Results:**

In [11]:
model.show_moments()

Equilibrium objects:
 rb: 0.0050
 ra: 0.0142
 w: 1.814
 Pi: 0.174
 capital-labor discrepancy: -0.00000998
 profit discrepancy: 0.00000313

Aggregates:
 GDP: 1.739
 capital-labor ratio: 28.158
 capital-output ratio: 2.340
 bond-output ratio: 0.233

Fractions:
 poor HtM: 0.102 [0.102, 0.146]
 wealthy HtM: 0.187 [0.187, 0.280]
 borrowers: 0.146 [0.146, 0.133]

Iliquid wealth:
 top   0.1: 0.071
 top     1: 0.406
 top    10: 0.877
 bottom 50: 0.001
 bottom 25: 0.000
 gini: 0.820

Liquid wealth:
 top   0.1: 0.024
 top     1: 0.182
 top    10: 0.750
 bottom 50: -0.032
 bottom 25: -0.032
 gini: 0.862

MPC: 0.157
Consumption: avarage = 1.146, gini = 0.311
Value: avarage = -18.956
